In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')
from src import AFDB_tools, treescore , foldseek2tree
import pandas as pd
import torch

/home/cactuskid/miniconda3/envs/ML2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
#load cath dataset from CATH data folder with comment lines as # and white space seperator



domain_df = pd.read_csv('../CATH_data/cath-domain-list.txt' , comment='#' , header = None , delim_whitespace=True)
print(domain_df.head())

        0   1   2   3   4   5   6   7   8   9   10     11
0  1oaiA00   1  10   8  10   1   1   1   1   1  59    1.0
1  1go5A00   1  10   8  10   1   1   1   1   2  69  999.0
2  3frhA01   1  10   8  10   2   1   1   1   1  58    1.2
3  3friA01   1  10   8  10   2   1   1   1   2  54    1.8
4  3b89A01   1  10   8  10   2   1   1   2   1  54    2.6


In [32]:
#change the type to string
domain_df = domain_df.astype(str)
domain_df['superfam'] = domain_df[[1,2,3,4]].apply(lambda x : '.'.join(x), axis = 1) 


In [25]:
siftsdf = pd.read_csv('../CATH_data/pdb_chain_cath_uniprot.csv', header=1 )


In [33]:
#merge with domain df on cath id
siftsdf = siftsdf.merge(domain_df, left_on = 'CATH_ID' , right_on = 0 , how = 'left')
siftsdf = siftsdf.dropna()
print(siftsdf.head(), len(siftsdf) , print(len(siftsdf.superfam.unique())))

438
    PDB CHAIN SP_PRIMARY  CATH_ID      0_x 1_x 2_x  3_x 4_x 5_x  ...    3   4  \
0  101m     A     P02185  101mA00  101mA00   1  10  490  10   1  ...  490  10   
1  102m     A     P02185  102mA00  102mA00   1  10  490  10   1  ...  490  10   
2  103m     A     P02185  103mA00  103mA00   1  10  490  10   1  ...  490  10   
3  104m     A     P02185  104mA00  104mA00   1  10  490  10   1  ...  490  10   
4  105m     A     P02185  105mA00  105mA00   1  10  490  10   1  ...  490  10   

   5  6  7   8   9   10    11     superfam  
0  1  1  1  51   4  154  2.07  1.10.490.10  
1  1  1  1  42   7  154  1.84  1.10.490.10  
2  1  1  1  42   8  154  2.07  1.10.490.10  
3  1  1  1   1  67  153  1.71  1.10.490.10  
4  1  1  1   1  88  153  2.02  1.10.490.10  

[5 rows x 69 columns] 311581 None


In [34]:
print(siftsdf.superfam.unique(),len(siftsdf.superfam.unique()))

['1.10.490.10' '3.40.30.10' '1.20.1050.10' '3.40.50.150' '3.30.930.10'
 '3.40.50.300' '3.10.200.10' '1.10.530.10' '1.10.760.10' '2.60.40.10'
 '3.90.10.10' '1.20.5.170' '1.10.238.10' '3.40.50.2300' '1.10.10.10'
 '3.40.718.10' '3.30.200.20' '1.10.510.10' '3.30.505.10' '4.10.280.10'
 '1.20.120.160' '3.20.20.150' '3.30.470.10' '3.20.10.10' '2.40.10.10'
 '3.30.1490.70' '2.40.50.140' '3.30.470.30' '3.30.450.30' '2.30.30.40'
 '1.10.150.130' '2.130.10.10' '2.120.10.10' '2.40.50.40' '3.40.350.10'
 '3.90.230.10' '1.25.40.10' '2.40.128.20' '3.30.160.60' '3.30.500.10'
 '2.40.10.120' '3.40.50.1370' '1.10.533.10' '1.20.1250.10' '2.60.40.150'
 '3.40.50.720' '1.10.565.10' '3.10.450.50' '3.65.10.10' '3.40.50.200'
 '2.40.50.90' '3.10.450.40' '2.40.70.10' '2.40.100.10' '3.40.50.2020'
 '3.20.20.80' '2.60.120.200' '2.10.25.10' '3.10.110.10' '3.20.20.60'
 '2.60.40.420' '3.40.190.10' '2.60.20.10' '2.60.40.1180' '3.30.470.20'
 '3.30.565.10' '3.40.50.10860' '3.20.20.140' '3.40.605.10' '3.40.309.10'
 '3.80.10.1

In [38]:
from src import AFDB_tools
import os
datapath = '../CATH_data/'
nprots = 250
#iterate over all superfamilies and create a tree for each

if not os.path.exists(datapath):
    os.mkdir(datapath)

for superfam in siftsdf.superfam.unique():

    if not os.path.exists(datapath+superfam):
        os.mkdir(datapath+superfam)
    
    #create a tree for each superfam
    #sample 1000 proteins from the superfam
    sub = siftsdf[siftsdf.superfam == superfam]
    prots = sub.sample(min(nprots, len(sub))).dropna()
    print(len(prots))

    #output the sequence dataset to a file
    prots.to_csv(datapath+superfam+'/sequence_dataset.csv')
    #create a folder for the superfam if it does not exist
    #output the uniport ids to a file
    with open(datapath+superfam+'/identifiers.txt', 'w') as f:
        f.write('\n'.join( list(prots['SP_PRIMARY'])))

250
250
250
250
250
250
250
250
250
250
126
250
250
250
250
250
250
250
250
98
48
250
250
250
250
64
250
129
189
250
90
250
250
250
192
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
193
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
105
250
250
250
163
250
250
250
250
250
250
250
130
250
250
245
250
250
190
250
137
250
250
250
120
250
250
250
250
250
250
250
250
250
250
160
250
250
250
250
250
250
250
250
250
209
70
250
91
250
250
250
156
206
250
250
250
149
193
231
250
240
250
250
250
250
250
250
69
250
250
250
250
250
250
124
210
250
250
250
250
250
250
76
250
28
174
250
250
231
250
66
75
250
84
236
73
181
250
250
250
83
250
250
141
250
250
165
250
250
250
250
234
193
250
250
250
250
250
205
73
250
250
250
250
68
250
250
250
250
250
181
143
246
214
250
199
108
250
250
250
250
250
250
195
250
250
250
250
227
250
250
169
250
250
250
250
250
250
168
54
250
250
117
250
250
250
250
250
250
250
250
250
244
250
250
9